In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import chromadb

ModuleNotFoundError: No module named 'chromadb'

In [3]:
url = "https://es.wikipedia.org/wiki/Geograf%C3%ADa_de_Colombia"

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# Buscar todas las tablas en la página
tables = soup.find_all('table')

# Verificar si alguna tabla contiene información sobre departamentos y capitales
tabla = None
for table in tables:
    if "Departamento" in table.get_text() and "Capital" in table.get_text():
        tabla = table
        break

# Si se encuentra la tabla, convertirla en un DataFrame
if tabla:
    df_departamentos = pd.read_html(str(tabla))[0]
    df_departamentos
else:
    print("No se encontró una tabla con información de departamentos y capitales.")

C:\Users\Victor\AppData\Local\Temp\ipykernel_16652\765528878.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_departamentos = pd.read_html(str(tabla))[0]


In [5]:
# Eliminar la columna 0 y la primera fila
df_departamentos = df_departamentos.iloc[1:, 1:]

In [6]:
new_column_names = df_departamentos.iloc[0]
df_departamentos.columns = new_column_names.values
df_departamentos = df_departamentos.iloc[1:]

In [7]:
modelo_nombre = 'all-MiniLM-L6-v2'
try:
    model = SentenceTransformer(modelo_nombre)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error: {e}")

c:\Users\Victor\Desktop\Productos\LLM-Wikipedia-search-region\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Victor\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Modelo cargado exitosamente.


In [9]:
documentos = []
for index, row in df_departamentos.iterrows():
    documento = f"Departamento: {row['Departamento']}, Capital: {row['Capital']}"
    documentos.append(documento)

In [11]:
try:
    embeddings = model.encode(documentos)
    print(f"Se generaron {len(embeddings)} embeddings.")
    print(f"Dimensión de cada embedding: {embeddings.shape[1]}")
except Exception as e:
    print(f"Error al generar los embeddings: {e}")

Se generaron 35 embeddings.
Dimensión de cada embedding: 384


In [ ]:
client = chromadb.Client()

# Para una base de datos persistente en disco:
# client = chromadb.PersistentClient(path="./mi_base_de_datos")

# Nombre de la colección donde guardarás tus embeddings
collection_name = "departamentos_capitales"

# Obtener o crear la colección
collection = client.get_or_create_collection(name=collection_name)

print(f"Colección '{collection_name}' creada o cargada.")

In [ ]:
ids = [f"doc_{i}" for i in range(len(documentos))]  # Generar IDs únicos para cada documento

collection.add(
    embeddings=embeddings,
    documents=documentos,
    ids=ids
)

print(f"Se han añadido {collection.count()} documentos a la colección '{collection_name}'.")